## Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distribution

import math
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import visualize
from flows import nf, nqf

import os, time
import argparse

## Parameters

In [2]:
parser = argparse.ArgumentParser()

# action
parser.add_argument('--train', action='store_true', help='Train a flow.')
parser.add_argument('--evaluate', action='store_true', help='Evaluate a flow.')
parser.add_argument('--plot', action='store_true', help='Plot a flow and target density.')
parser.add_argument('--restore_file', type=str, help='Path to model to restore.')
parser.add_argument('--output_dir', default='.', help='Path to output folder.')
parser.add_argument('--no_cuda', action='store_true', help='Do not use cuda.')

# target potential
parser.add_argument('--target_potential', choices=['u_z0', 'u_z5', 'u_z1', 'u_z2', 'u_z3', 'u_z4'], help='Which potential function to approximate.')

# flow params
parser.add_argument('--base_sigma', type=float, default=4, help='Std of the base isotropic 0-mean Gaussian distribution.')
parser.add_argument('--learn_base', default=False, action='store_true', help='Whether to learn a mu-sigma affine transform of the base distribution.')
parser.add_argument('--flow_length', type=int, default=2, help='Length of the flow.')

# training params
parser.add_argument('--init_sigma', type=float, default=1, help='Initialization std for the trainable flow parameters.')
parser.add_argument('--batch_size', type=int, default=100)
parser.add_argument('--start_step', type=int, default=0, help='Starting step (if resuming training will be overwrite from filename).')
parser.add_argument('--n_steps', type=int, default=1000000, help='Optimization steps.')
parser.add_argument('--lr', type=float, default=1e-5, help='Learning rate.')
parser.add_argument('--weight_decay', type=float, default=1e-3, help='Weight decay.')
parser.add_argument('--beta', type=float, default=1, help='Multiplier for the target potential loss.')
parser.add_argument('--seed', type=int, default=2, help='Random seed.')

args = parser.parse_args("--train --target_potential u_z0 --flow_length 2 --n_steps 30000".split())
args.device = torch.device('cuda:1')

## Inference tasks

In [39]:
## [A]. Gaussian problem (for sanity check)
mu = 1.5 + (torch.zeros(2)).to(args.device)
cov = torch.tensor([[1.2, 0.55],[0.55, 1.2]]).to(args.device)
u_normal = distribution.MultivariateNormal(mu, cov)
u_z0 = lambda z: -u_normal.log_prob(z)
u_z0_sampler = lambda n: u_normal.sample([n])

In [4]:
## [B]. G-and-K with diff params (used to show the expressive power of NQF)
def random_flow_param(flow):
    GK = flow.transforms[0]
    GK.A_.data.fill_(0)
    GK.B_.data.fill_(2.0*torch.randn(1,1).item())  
    GK.g_.data.fill_(-0.75 + 1.5*torch.rand(1,1).item())
    GK.k_.data = (-1 + 2*torch.randn(GK.k_.size())-0.69315).data
    GK.C_.data = (torch.randn(GK.C_.size())).data
    
n_show = 8   
for i in range(n_show):
    flow = nqf.NeuralQuantileFlow(dim=2, flow_length=args.flow_length, inversion=None)
    random_flow_param(flow)
    flow.to((args.device))
    visualize.plot_flow_density(flow, plt.gca(), device=args.device, output_file='./example/results_ncf_{}.png'.format(i))

In [5]:
## [C]. 2D approx log-normal problem
class ApproxLogNormal:
    
    def __init__(self, mus, sigmas, rho):
        self.mus = torch.tensor(mus).view(1, 2)
        self.sigmas = torch.tensor(sigmas).view(1, 2)
        self.rho = torch.tensor(rho).view(1, 1)
        self.s = 1.0
        self.t = 0.0
        self.k = 1.1
        
    def to(self, device):
        self.device = device
        self.mus = self.mus.to(device)
        self.sigmas = self.sigmas.to(device)
        self.rho = self.rho.to(device)
        mu = self.mus.view(-1).to(self.device)
        sigma1, sigma2, rho = self.sigmas[0,0].item(), self.sigmas[0,1].item(), self.rho.item()
        cov = torch.tensor([[sigma1**2, rho*sigma1*sigma2],[rho*sigma1*sigma2, sigma2**2]]).to(self.device)
        self.normal = distribution.MultivariateNormal(mu, cov)
        return self
            
    def log_prob(self, x):
        n = len(x)
        y = (x-1).sign()/2 + 0.5
        z1, z2 = 1-(1-x).exp(), x.abs().log()        
        dx_z = torch.zeros(x.size()).requires_grad_(True).to(x.device)
        dx_z1, dx_z2 = 1/(1-z1), z2.exp()
        z = (1-y)*z1 + y*z2
        dx_z = (1-y)*dx_z1 + y*dx_z2
        base_prob = self.normal.log_prob(z).view(n, -1)
        log_abs_det_jacobian = torch.sum(dx_z.log(), dim=1).view(n, -1)
        return (base_prob - log_abs_det_jacobian).view(-1)
    
    def sample(self, N):
        n = N[0]
        s, t, k = self.s, self.t, self.k
        z = self.normal.sample([n])
        y = z.sign()/2 + 0.5
        z1, z2 = 1-(-z+1).log(), z.exp()
        x = (1-y)*z1 + y*z2
        return x
        
alog_normal = ApproxLogNormal(mus=[0.1, 0.1], sigmas=[0.5, 0.5], rho=0.4).to(args.device)
u_z2 = lambda z: -alog_normal.log_prob(z)
u_z2_sampler = lambda n: alog_normal.sample([n])
visualize.plot_target_density(u_z2, plt.gca(), z_range=[-0.1, 3], device=args.device,output_file='./alog_normal_density_rho{:.1f}.png'.format(alog_normal.rho.item()))

## Train

In [7]:

def train(flow, target_energy_potential, target_energy_potential_sampler, optimizer, args):
    '''
        minimizing KL[q(z)||p(z)] 
    '''
    for i in range(args.start_step, args.n_steps):
        # sample z ~ q(z)
        n = 100 if i<args.n_steps-1 else 1000
        z = flow.base_dist.sample((100, )).to(args.device)

        # pass through flow
        base_log_prob = flow.base_dist.log_prob(z)
        zk, sum_log_abs_det_jacobians = flow(z)
        q_log_prob = base_log_prob - sum_log_abs_det_jacobians
        
        # the target energy
        p_log_prob = - target_energy_potential(zk)  # p = exp(-potential) ==> p_log_prob = - potential
        
        # compute loss and optimize
        loss = q_log_prob - p_log_prob
        loss = loss.mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # report
        if i%int(args.n_steps/5) == 0 or i==args.n_steps-1: 
            print('finish {}/{}'.format(i, args.n_steps), 'loss=', loss.item())
            flow.print()
            print('|p(x)-q(x)|=', torch.abs(p_log_prob - q_log_prob).mean().item())
            print('\n')
    return


flow = nqf.NeuralQuantileFlow(dim=2, flow_length=1, inversion=None).to(args.device)
file = 'exp/test/nqf_k{}.model'.format(flow.length)

optimizer = torch.optim.Adam(flow.parameters(), lr=0.001, weight_decay=1e-5)
u_z, u_z_sampler = u_z2, u_z2_sampler
if True:
    train(flow, u_z, u_z_sampler, optimizer, args)
    torch.save(flow.state_dict(), file)
else:
    state = torch.load(file)
    flow.load_state_dict(state)

finish 0/30000 loss= 236.32980346679688
A= tensor([[0.0010, 0.0010]], device='cuda:1')
B= tensor([[0.9990, 0.9990]], device='cuda:1')
g= tensor([[0.0010, 0.0010]], device='cuda:1')
k= tensor([[-0.0005, -0.0005]], device='cuda:1')
c= tensor([[0.6703, 0.6703]], device='cuda:1')
V= tensor([[1.0000e+00, 1.0000e-03],
        [1.0000e-03, 1.0000e+00]], device='cuda:1')
|p(x)-q(x)|= 236.6287384033203


finish 6000/30000 loss= 0.042456887662410736
A= tensor([[1.0951, 1.0960]], device='cuda:1')
B= tensor([[0.6197, 0.6225]], device='cuda:1')
g= tensor([[0.3626, 0.3566]], device='cuda:1')
k= tensor([[-0.1139, -0.1127]], device='cuda:1')
c= tensor([[0.7596, 0.7652]], device='cuda:1')
V= tensor([[1.0000, 0.3981],
        [0.3981, 1.0000]], device='cuda:1')
|p(x)-q(x)|= 0.20722712576389313


finish 12000/30000 loss= 0.03231219947338104
A= tensor([[1.1343, 1.1426]], device='cuda:1')
B= tensor([[0.5740, 0.5805]], device='cuda:1')
g= tensor([[0.5156, 0.5026]], device='cuda:1')
k= tensor([[0.0487, 0.032

## Visualize

In [8]:
visualize.plot_target_density(u_z, plt.gca(), z_range=[-0.1, 3], device=args.device, output_file='./results_target_density.png')
visualize.plot_flow_density(flow, plt.gca(), z_range=[-0.1, 3], device=args.device, output_file='./results_nqf_k{}_density.png'.format(flow.length))